In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
"""data = np.stack([temp,pressure,humidity],axis = -1)
scalar = MinMaxScaler()
data = scalar.fit_transform(data)
data.respahe(num_samples,sequence_len ,features)

X = 
Y = 

In [17]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def generate_syn_data(sequence_length=100, num_samples=100):
    time = np.arange(0, num_samples * sequence_length) / 100
    temp = 20 + 10 * np.sin(0.1 * time)
    humidity = 50 + 10 * np.cos(0.05 * time)
    pressure = 1000 + 10 * np.sin(0.02 * time)

    # Stack features into a 2D array (samples * sequence_length, features)
    data = np.stack([temp, humidity, pressure], axis=-1)

    # Scale the data (MinMaxScaler expects a 2D array)
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)

    # Reshape into 3D array (num_samples, sequence_length, features)
    data = data.reshape(num_samples, sequence_length, 3)

    # Split into input (X) and target (Y)
    X = data[:, :-1, :]  # All time steps except the last
    Y = data[:, -1, :]   # Last time step

    return X, Y, scaler

# Generate data
X, Y, scaler = generate_syn_data()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

In [ ]:
"""def generate_syn_data(sequence_length = 100, num_samples = 100):
    time = np.arange(0,num_samples * sequence_length)/100
    temp = 20 + 10 * np.sin(0.1 * time)

    humidity = 50 + 10 * np.cos(0.05 * time)
    pressure = 1000 + 10*np.sin(0.02 * time)

    data = np.stack([temp,humidity,pressure],axis = -1)
    data = data.reshape(num_samples, sequence_length,3)

    scaler = MinMaxScaler()
    data = scaler.fit_transform(data.reshape(-1,3).reshape(num_samples,sequence_length,3))

    
    X = data[:, :-1, :]
    Y = data[:, -1, :]

    return X, Y, scaler""""""

In [8]:
class RNNModel(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers = 1):
        super(RNNModel,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size,hidden_size,num_layers,batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self,x):
        h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size)
        out,_ = self.rnn(x,h0)
        out = self.fc(out[:,-1,:])
        return out     

In [26]:
class LSTMModel(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers = 1):
        super(LSTMModel,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,num_layers,batch_first = True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size)
        c0 = torch.zeros(self.num_layers,x.size(0),self,hidden_size)

        out,_ = self.lstm(x,(h0,c0))
        out = self.fc(out[:,-1,:])
        return out


        


In [19]:
X,Y,scaler = generate_syn_data()

X_tensor = torch.tensor(X,dtype = torch.float32)
Y_tensor = torch.tensor(Y,dtype = torch.float32)

In [27]:
input_size = 3
hidden_size = 50
output_size = 3
num_layers = 1
learning_rate = 0.01

model = LSTMModel(input_size,hidden_size,output_size,num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [25]:
input_size = 3  # Number of features (temperature, humidity, pressure)
hidden_size = 50  # Number of hidden units in LSTM
output_size = 3  # Number of output features (temperature, humidity, pressure)
num_layers = 1  # Number of LSTM layers
learning_rate = 0.01
num_epochs = 100

# Initialize model, loss, and optimizer
model = LSTMModel(input_size, hidden_size, output_size, num_layers)
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_tensor)  # Model predictions
    loss = criterion(outputs, Y_tensor)  # Compute loss

    # Backward pass and optimization
    optimizer.zero_grad()  # Clear gradients
    loss.backward()  # Compute gradients
    optimizer.step()  # Update weights

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")


AttributeError: 'LSTMModel' object has no attribute 'num_layers'

In [28]:
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_tensor)
    loss = criterion(outputs,Y_tensor)

    loss.backward()
    optimizer.step()

    if(epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}],Loss { loss.item():.4f}')

print("Training Complete")
print("Accuracy is ", (1-loss)*100,"%")
    

TypeError: zeros(): argument 'size' failed to unpack the object at pos 3 with error "type must be tuple of ints,but got LSTMModel"

In [14]:
a = np.arange(0,100*100)/100

In [15]:
print(a)

[0.000e+00 1.000e-02 2.000e-02 ... 9.997e+01 9.998e+01 9.999e+01]


In [26]:
a = np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
b = np.array([[[9,10],[11,12]],[[13,14],[15,16]]])

print(f"shape of a, {a.shape},shape of b{b.shape}")

shape of a, (2, 2, 2),shape of b(2, 2, 2)


In [17]:
F = np.stack([a,b],axis = 0)
print(F)

[[[[ 1  2]
   [ 3  4]]

  [[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]

  [[13 14]
   [15 16]]]]


In [18]:
Q = np.stack([a,b],axis = 1)
print(Q)


[[[[ 1  2]
   [ 3  4]]

  [[ 9 10]
   [11 12]]]


 [[[ 5  6]
   [ 7  8]]

  [[13 14]
   [15 16]]]]


In [27]:
G = np.stack([a,b],axis = 2)
print(G)

[[[[ 1  2]
   [ 9 10]]

  [[ 3  4]
   [11 12]]]


 [[[ 5  6]
   [13 14]]

  [[ 7  8]
   [15 16]]]]


In [19]:
R = np.stack([a,b],axis = 3)
print(R)

[[[[ 1  9]
   [ 2 10]]

  [[ 3 11]
   [ 4 12]]]


 [[[ 5 13]
   [ 6 14]]

  [[ 7 15]
   [ 8 16]]]]


In [21]:
a,b,c,d = 2,5,7,9

In [22]:
A0 = np.random.random((a, b, c, d))
B0 = np.random.random((a, b, c, d))

In [24]:
print(f'Matrix A0 {A0}')
print(f'Matrix B0 {B0}')
W = np.stack([A0,B0],axis = 4)

Matrix A0 [[[[6.08866963e-01 6.84034612e-01 1.47939524e-01 1.31607330e-01
    9.05533369e-01 8.57729637e-01 5.44710645e-01 8.05065792e-01
    7.14253206e-01]
   [7.42630308e-01 1.70884808e-01 7.07726130e-01 4.23926358e-01
    8.38011881e-02 4.41398816e-01 7.35703129e-02 7.67007261e-01
    8.16387041e-01]
   [5.02086574e-01 5.78301783e-01 3.74474056e-01 2.52973241e-01
    1.18123059e-01 6.75227735e-01 9.15339447e-01 8.62753763e-01
    9.77878298e-01]
   [1.09106840e-01 3.62422459e-01 2.32074320e-01 7.75803676e-01
    6.28114149e-01 7.06124420e-01 7.26821199e-01 3.27319456e-01
    1.62294752e-01]
   [4.80434022e-01 5.35426265e-01 8.85665188e-01 6.08677654e-01
    1.87986027e-01 7.63633123e-01 2.57619650e-01 2.00007899e-01
    6.33759960e-01]
   [1.70382055e-01 9.26016327e-01 6.79310618e-01 6.80151787e-01
    1.75696434e-01 9.65088286e-01 7.43252113e-01 2.23319585e-01
    6.38931828e-01]
   [1.16400344e-01 2.56026187e-01 5.20779292e-01 2.53513463e-01
    5.44488125e-02 5.06252033e-01 8.57